# Real-Time CNN Susceptibility Artifact Detection
Using nnU-Net trained model.

In [1]:
"""
https://github.com/MIC-DKFZ/nnUNet/blob/master/nnunetv2/inference/examples.py
"""

if __name__ == '__main__':
    from nnunetv2.paths import nnUNet_results, nnUNet_raw
    import torch
    from batchgenerators.utilities.file_and_folder_operations import join
    from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor
    from nnunetv2.imageio.simpleitk_reader_writer import SimpleITKIO
    import cv2
    import os
    import time
    import numpy as np

    # nnUNetv2_predict -d 3 -f 0 -c 3d_lowres -i imagesTs -o imagesTs_predlowres --continue_prediction
    
    # nnUNetv2_predict.exe -i .\imagesTr\ -o .\RESULTS\ -d 1 -c 2d -f 4

    # instantiate the nnUNetPredictor
    predictor = nnUNetPredictor(
        tile_step_size=1,
        use_gaussian=True,
        use_mirroring=True,
        perform_everything_on_gpu=True,
        device=torch.device('cuda', 0),
        verbose=True,
        verbose_preprocessing=True,
        allow_tqdm=True
    )
    # initializes the network architecture, loads the checkpoint
    predictor.initialize_from_trained_model_folder("MODEL", checkpoint_name='checkpoint_final.pth', use_folds=(4,))
    
    """
    a np.ndarray of shape (c, x, y, z) where c is the number of image channels (can be 1) and x, y, z are the spatial dimensions (set x=1 for 2D! Example: (3, 1, 224, 224) for RGB image). 2) a dictionary with metadata. This can be anything. BUT it HAS to include a {'spacing': (a, b, c)} where a is the spacing of x, b of y and c of z! If an image doesn't have spacing, just set this to 1. For 2D, set a=999 (largest spacing value! Make it larger than b and c
    
    In the case of 2D images, the returned array should have shape (c, 1, x, y) and the spacing should be (999, sp_x, sp_y). Make sure 999 is larger than sp_x and sp_y! Example: shape=(3, 1, 224, 224), spacing=(999, 1, 1)
    For images that don't have a spacing, set the spacing to 1 (2d exception with 999 for the first axis still applies!)#
    predict a single numpy array
    """
    cv2.namedWindow("Prediction", cv2.WINDOW_AUTOSIZE)
    cv2.namedWindow("Image", cv2.WINDOW_AUTOSIZE)
    while True:
        for test_image in os.listdir("MODEL/test-images"):
            image = cv2.imread(os.path.join("MODEL/test-images", test_image), cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (512, 512)).astype(np.float32) / 255.0
            cnn_input = image.reshape(1, 1, image.shape[0], image.shape[1])
            props = {'spacing': (999, 1, 1)}
            output = predictor.predict_single_npy_array(cnn_input, props, None, None, True)[0]
            
            output_display = (output * 255).astype(np.uint8)
            image_display = (image * 255).astype(np.uint8)
            cv2.imshow("Prediction", output_display)
            cv2.imshow("Image", image_display)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cv2.destroyAllWindows()
    

nnUNet_raw is not defined and nnU-Net can only be used on data for which preprocessed files are already present on your system. nnU-Net cannot be used for experiment planning and preprocessing like this. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up properly.
nnUNet_preprocessed is not defined and nnU-Net can not be used for preprocessing or training. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up.
nnUNet_results is not defined and nnU-Net cannot be used for training or inference. If this is not intended behavior, please read documentation/setting_up_paths.md for information on how to set this up.


ModuleNotFoundError: No module named 'cv2'